# Model Inference Utilities
> Functions that are used the run a model over the test set

In [ ]:
#hide
# default_exp inference_utils
from cellsegment.core import *
from nbdev.showdoc import *

In [ ]:
# export
from fastai import *
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *

import cv2
from skimage.measure import label, regionprops

from skimage import filters
from skimage.morphology import erosion, dilation, opening, closing, disk
from scipy.spatial import distance
import pandas as pd
from pandas import DataFrame

# %% md
## Draw text and graphics on an matplotlib image

In [ ]:
# export
def bb_hw(a): return np.array([a[1], a[0], a[3] - a[1], a[2] - a[0]])

def draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(
        linewidth=lw, foreground='black'), patheffects.Normal()])

def draw_rect(ax, b):
    patch = ax.add_patch(patches.Rectangle(b[:2], *b[-2:], fill=False, edgecolor='white', lw=2))
    draw_outline(patch, 4)

def draw_text(ax, xy, txt, sz=12, color='white', valign='top'):
    text = ax.text(*xy, txt,
                   verticalalignment=valign, color=color, fontsize=sz)  # , weight='bold')
    draw_outline(text, 1)

# %%
# export
def run_inferences(learn:Learner, fnames:list, start:int=0, number_files=20) -> list:
    """
    run inferences over a list of `fnames` using `learner`  
    - return a list of predictions
    """
    preds = []
    if number_files == 'all':
        __number_files = len(fnames)
    elif isinstance(number_files, int) :
        __number_files = number_files
    else:
        __number_files = 0
    for i in range(start, start+__number_files):
        if i >= len(fnames): break
        img = open_image(fnames[i])
        y, pred, raw_pred = learn.predict(img)
        preds.append({"filename": fnames[i].name, "y": y,"pred": pred,"raw_pred": raw_pred})
    return preds



def plot_inferences(preds:List, df:DataFrame, src_path:Path=Path(''), start:int=0, rows:int = 4, cols:int=5) -> None:
    """
    Plot test images with auto markup labels
    """
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    ax_lst = []
    for i, ax in enumerate(axes.flat):
        if i+start >= len(preds): break
        fpath = src_path/preds[i+start]["filename"]
        img = open_image(fpath)
        width, height = img.size
        show_image(img, ax=ax)
        show_image(preds[i+start]["y"], ax=ax,  cmap='tab20', alpha=0.5)
        fn = Path(fpath).name
        # B,L,R,O = df.loc[fn, 'Background'], df.loc[fn, 'Fluke Liver'], df.loc[fn, 'Fluke Rumen'],df.loc[fn, 'Other']

        pstr = df.loc[df['Name'] == fn, 'Pstr'].values[0]
        draw_text(ax, (0, 0), f'{fpath.name}', sz=14, color='red')
        draw_text(ax, (0, height-20), pstr, sz=14, color='red')

        ax.set_title(f'File {fpath.name} \n {pstr}')
        ax_lst.append(ax)

    plt.tight_layout()
    return None


def create_probs_df(fnames: List) -> DataFrame:
    """
    create a data frame of prob from a `fname` list
     - param fnames: 
     - return: `DataFrame`
    """
    list_filedata = [None] * len(fnames)
    for i, fn in enumerate(fnames):
        # img = PIL.Image.open(fn)
        # img_w, img_h = img.size
        list_filedata[i] = {'Name': fn.name, 'Background': '', 'Fluke Liver': '', 'Fluke Rumen': '', 'Other': '', 'Pstr': ''}

    df = pd.DataFrame(list_filedata)
    df = df[['Name', 'Background', 'Fluke Liver', 'Fluke Rumen', 'Other', 'Pstr']]
    df.set_index("Name", inplace=True)
    return df


def add_cols_to_probs_df(df: DataFrame) -> DataFrame:
    add_col = [''] * df.shape[0]
    df['Background'] = add_col
    df['Fluke_Liver'] = add_col
    df['Fluke_Rumen'] = add_col
    df['Other'] = add_col
    df['Pstr'] = add_col
    return df


def probs_to_df(preds:List, df:DataFrame):
    for i, pred in enumerate(preds):
        offset = 0
        # if i >= len(preds): break
        fn = Path(pred["filename"]).name
        # fn = preds[i+offset]["filename"]
        probs = calc_probs(pred["raw_pred"])
        # problist.append()pstr
        if len(probs) > 0:
            pstr = f"B:{probs[0]:3.2f} L:{probs[1]:3.2f} R:{probs[2]:3.2f}"
            df.loc[df['Name'] == fn, ['Background', 'Fluke_Liver', 'Fluke_Rumen', 'Other', 'Pstr']] = probs+[pstr]
            #
            # df.loc[fn, 'Background'] = probs[0]
            # df.loc[fn, 'Fluke Liver'] = probs[1]
            # df.loc[fn, 'Fluke Rumen'] = probs[2]
            # df.loc[fn, 'Other'] = probs[3]
            # df.loc[fn, 'Pstr'] = pstr


def __get_props(raw_pred:tensor, layer:int, min_conf:float=0.2, min_area:int = 100) -> list:

    arr = raw_pred[layer, :, :].numpy()
    if layer == 0:
        label_arr = label(arr < 1 - min_conf)
    else:
        label_arr = label(arr > min_conf)

    region_props = []
    for region in regionprops(label_arr, arr, cache=True):
        if region.area > min_area:
            cx = int(region.centroid[1])
            cy = int(region.centroid[0])
            region_props.append({
                "class_layer": layer,
                "centroid": region.centroid,
                "mean_intensity": region.mean_intensity,
                "area": region.area,
                "coords": region.coords,
            })
    return region_props


def calc_probs(raw_pred:tensor)->list:
    """
    Calculate the probabilities from a raw prediction  
    by finding the average value of all the pixels within a region  
    This assumes that there is only one cell in the image
    :param raw_pred:   
    :return:   
    """
    def __get_bool_mask(o):
        min_conf = 0.5
        arr = o[0, :, :].numpy()
        bgnd = __get_props(o, 0, min_conf)
        if len(bgnd) > 0:
            mask = np.zeros((arr.shape), dtype=bool)
            AAcoords = bgnd[0]["coords"]
            mask[tuple(AAcoords.T.tolist())] = True
        else:
            mask = None
        return mask    
    
    mask = __get_bool_mask(raw_pred)
    if mask is not None:
        p = [None] * raw_pred.shape[0]
        for i in range(raw_pred.shape[0]):
            p[i] = raw_pred[i, :, :].numpy()[mask].mean().round(2)
    else:
        p = []
    return p



